In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import re
import string
import json
import sys
#import classificationreport
from sklearn.metrics import classification_report, confusion_matrix
from spacy.lang.en import English
eng = English()
tok = eng.tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#NLI_LSTM with 3 class classification with dense layer
class NLI_LSTM(nn.Module):
    def __init__(self,embedding_matrix, vocab_size, embedding_dim, hidden_dim, output_dim,n_fc_layers, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.embedding.weight.requires_grad = False
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,batch_first=True)
        self.fc=nn.ModuleList([nn.Linear(hidden_dim * 2,hidden_dim * 2) for i in range(n_fc_layers)])
        self.linear=nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    
    # forward pass
    # NLI with 2 text inputs
    def forward(self, text1, text2):
        #text = [sent len, batch size]
        embedded1 = self.dropout(self.embedding(text1))
        embedded2 = self.dropout(self.embedding(text2))
        #embedded = [sent len, batch size, emb dim]
        output1, (hidden1, cell1) = self.lstm(embedded1)
        output2, (hidden2, cell2) = self.lstm(embedded2)

        hidden = torch.cat((hidden1,hidden2),dim=2)
        hidden = torch.squeeze(hidden)

        for i in range(len(self.fc)-1):
            hidden = self.fc[i](hidden)
            hidden = F.relu(hidden)
            hidden = self.dropout(hidden)
        
        hidden = self.fc[len(self.fc)-1](hidden)
        hidden = F.log_softmax(hidden,dim=1)
        hidden = self.dropout(hidden)

        hidden = self.linear(hidden)
        #hidden = [batch size, output dim]
        return hidden

In [3]:
#load glove vectors
embeddings_index = {}
f = open('../data/glove.6B.300d.txt')
lines = f.readlines()
for line in tqdm(lines):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

100%|████████████████████████████████| 400000/400000 [00:24<00:00, 16257.27it/s]


In [4]:
# tokenise input and remove punctuation and numbers using only regex
def tokenize(text):
    # text = re.sub(r'[^a-zA-Z ]', '', text)
    # #convert to lowercase
    # text = text.lower()
    # return text.split()
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunctext = regex.sub(" ", text.lower())
    return [token.text for token in tok(nopunctext)]

UNK="<UNK>"
PAD="<PAD>"

#import data
def getDataset(dataset_name="mnli"):
    if dataset_name=="mnli":
        filepath_train="../data/multinli_1.0/multinli_1.0/multinli_1.0_train.jsonl"
        filepath_dev="../data/multinli_1.0/multinli_1.0/multinli_1.0_dev_matched.jsonl"
        filepath_test="../data/multinli_1.0/multinli_1.0/multinli_1.0_dev_mismatched.jsonl"
    elif dataset_name=="snli":
        filepath_train="../data/snli_1.0/snli_1.0/snli_1.0_train.jsonl"
        filepath_dev="../data/snli_1.0/snli_1.0/snli_1.0_dev.jsonl"
        filepath_test="../data/snli_1.0/snli_1.0/snli_1.0_test.jsonl"
    else:
        print("Invalid dataset name")
        return None
    
    #read train,dev and test data
    labels = ["contradiction", "entailment", "neutral"]
    f= open(filepath_train, "r")
    data = list(f)
    train_dataset={"premise":[],"hypothesis":[],"label":[]}
    print("train data")
    for line in tqdm(data):
        line = json.loads(line)
        if line['gold_label'] not in labels:
            # print(line['gold_label'])
            continue
        train_dataset["premise"].append(line['sentence1'])
        train_dataset["hypothesis"].append(line['sentence2'])
        train_dataset["label"].append(line['gold_label'])
    f.close()

    f= open(filepath_dev, "r")
    data = list(f)
    dev_dataset={"premise":[],"hypothesis":[],"label":[]}
    print("dev data")
    for line in tqdm(data):
        line = json.loads(line)
        if line['gold_label'] not in labels:
            # print(line['gold_label'])
            continue
        dev_dataset["premise"].append(line['sentence1'])
        dev_dataset["hypothesis"].append(line['sentence2'])
        dev_dataset["label"].append(line['gold_label'])
    f.close()

    f= open(filepath_test, "r")
    data = list(f)
    test_dataset={"premise":[],"hypothesis":[],"label":[]}
    print("test data")
    for line in tqdm(data):
        line = json.loads(line)
        if line['gold_label'] not in labels:
            # print(line['gold_label'])
            continue
        test_dataset["premise"].append(line['sentence1'])
        test_dataset["hypothesis"].append(line['sentence2'])
        test_dataset["label"].append(line['gold_label'])
    f.close()
    
    return train_dataset,dev_dataset,test_dataset

def getWord2index(dataset):
    word2index = {"":0,UNK:1,PAD:2}
    for sentence in dataset["premise"]:
        for word in tokenize(sentence):
            if word not in word2index:
                word2index[word] = len(word2index)
    for sentence in dataset["hypothesis"]:
        for word in tokenize(sentence):
            if word not in word2index:
                word2index[word] = len(word2index)
    return word2index

def getEmbeddingMatrix(word2index,emb_size=300):
    embedding_matrix = np.zeros((len(word2index),emb_size),dtype=np.float32)
    for word, i in word2index.items():
        if i==0:
            embedding_matrix[i] = np.zeros(emb_size)
        elif word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
        else:
            embedding_matrix[i] = np.random.uniform(-0.25,0.25,emb_size)
    return embedding_matrix

def getLabel2index(dataset):
    label2index = {"entailment":0,"neutral":1,"contradiction":2}
    return label2index

def getSentence2vector(sentence,word2index,padLength=32):
    sentence = tokenize(sentence)
    vector = []
    for word in sentence:
        if word in word2index:
            vector.append(word2index[word])
        else:
            vector.append(word2index[UNK])
    
    if len(vector)>padLength:
        vector=vector[:padLength]
    else:
        for i in range(padLength-len(vector)):
            vector.append(word2index[PAD])
    
    if(len(vector)!=padLength):
        print("Error in vector length")
    return np.array(vector)   

In [5]:
def preprocess(dataset,word2index,label2index):
    dataset["premise"] = [getSentence2vector(sentence,word2index) for sentence in tqdm(dataset["premise"])]
    dataset["hypothesis"] = [getSentence2vector(sentence,word2index) for sentence in tqdm(dataset["hypothesis"])]
    dataset["label"] = [label2index[label] for label in dataset["label"]]
    return dataset

def getDataloader(dataset,batch_size=32):
    # print(len(dataset["premise"]))
    premise = torch.tensor(dataset["premise"],dtype=torch.long)
    hypothesis = torch.tensor(dataset["hypothesis"],dtype=torch.long)
    labels = torch.tensor(dataset["label"],dtype=torch.long)
    dataset = torch.utils.data.TensorDataset(premise,hypothesis,labels)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

def prepData(dataset_name="mnli"):
    train_dataset,dev_dataset,test_dataset = getDataset(dataset_name)
    word2index = getWord2index(train_dataset)
    label2index = getLabel2index(train_dataset)
    embedding_matrix = getEmbeddingMatrix(word2index)
    #preprocess datasets
    print("preprocess")
    train_dataset = preprocess(train_dataset,word2index,label2index)
    dev_dataset = preprocess(dev_dataset,word2index,label2index)
    test_dataset = preprocess(test_dataset,word2index,label2index)
    #dataloader
    train_dataloader = getDataloader(train_dataset)
    dev_dataloader = getDataloader(dev_dataset)
    test_dataloader = getDataloader(test_dataset)

    return train_dataloader,dev_dataloader,test_dataloader,embedding_matrix,word2index


In [6]:
def train(model,train_dataloader,dev_dataloader,test_dataloader,optimizer,criterion,datasetName,epochs=5):
    
    
    f=open(f"../reports_and_results/report_lstm_nli_{datasetName}_foreachepoch.txt",'w')
    
    total_loss=[]
    total_acc=[]
    for epoch in range(1,1+epochs):
        ep_loss=0
        ep_acc=0
        train_total=0
        val_total=0
        test_total=0
        print("\n\nEpoch: ",epoch)
        #training
        print("Training")
        for batch in tqdm(train_dataloader):
            prem, hyp, label = batch
            # print(len(prem),len(hyp),len(label))
            optimizer.zero_grad()
            prem, hyp, label= prem.to(device), hyp.to(device), label.to(device)
            output = model(prem, hyp)
            # print(len(output))
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            ep_loss+=loss.item()
            total_loss.append(loss.item())
            acc = (output.argmax(1) == label).sum().item()
            train_total+=label.size(0)
            ep_acc+=acc
        total_acc.append((ep_acc/train_total))
        
        print("Validation")
        #validation
        val_y_true = []
        val_y_pred = []
        with torch.no_grad():
            ep_val_loss=0
            ep_val_acc=0
            for batch in tqdm(dev_dataloader):
                prem, hyp, label = batch
                prem, hyp, label= prem.to(device), hyp.to(device), label.to(device)
                output = model(prem, hyp)
                loss = criterion(output, label)
                val_y_true.extend(label.cpu())
                val_y_pred.extend(output.cpu().argmax(1))
                ep_val_loss+=loss.item()
                acc = (output.argmax(1) == label).sum().item()
                val_total+=label.size(0)
                ep_val_acc+=acc
        
        print("Test")
        y_true = []
        y_pred = []
        with torch.no_grad():
            ep_test_loss=0
            ep_test_acc=0
            for batch in tqdm(test_dataloader):
                prem, hyp, label = batch
                prem, hyp, label= prem.to(device), hyp.to(device), label.to(device)
                output = model(prem, hyp)
                y_true.extend(label.cpu())
                y_pred.extend(output.cpu().argmax(1))
                loss = criterion(output, label)
                ep_test_loss+=loss.item()
                acc = (output.argmax(1) == label).sum().item()
                test_total+=label.size(0)
                ep_test_acc+=acc
                
        print("Epoch: ",epoch," Train Loss: ",ep_loss/len(train_dataloader),"Train Accuracy: ",ep_acc/train_total)

        print("Epoch: ",epoch," Val Loss: ",ep_val_loss/len(dev_dataloader)," Val Accuracy: ",ep_val_acc/val_total)

        print("Epoch: ",epoch,"Test Loss: ",ep_test_loss/len(test_dataloader)," Test Accuracy: ",ep_test_acc/test_total)

        print()
        print("Validation Classification report")

        print(classification_report(val_y_true, val_y_pred, target_names=["entailment","neutral","contradiction"]))
        print("Test Classification report")
        print(classification_report(y_true, y_pred, target_names=["entailment","neutral","contradiction"]))
                
        orginal_stdout=sys.stdout
        sys.stdout=f 
        print("\n\nEpoch: ",epoch)

        
        print("Epoch: ",epoch," Train Loss: ",ep_loss/len(train_dataloader),"Train Accuracy: ",ep_acc/train_total)

        print("Epoch: ",epoch," Val Loss: ",ep_val_loss/len(dev_dataloader)," Val Accuracy: ",ep_val_acc/val_total)

        print("Epoch: ",epoch,"Test Loss: ",ep_test_loss/len(test_dataloader)," Test Accuracy: ",ep_test_acc/test_total)

        print()
        print("Validation Classification report")

        print(classification_report(val_y_true, val_y_pred, target_names=["entailment","neutral","contradiction"]))
        print("Test Classification report")
        print(classification_report(y_true, y_pred, target_names=["entailment","neutral","contradiction"]))
        #save model
        torch.save(model.state_dict(), f"../models/model_lstm_nli_{datasetName}_ep_{epoch}.pt")
        sys.stdout=orginal_stdout
        
    sys.stdout=f
    print("Total Train Loss",sum(total_loss)/len(total_loss)," Total Train Accuracy: ",sum(total_acc)/len(total_acc))
              
    sys.stdout=orginal_stdout
    print("Total Train Loss",sum(total_loss)/len(total_loss)," Total Train Accuracy: ",sum(total_acc)/len(total_acc))
    f.close()
    return model
    

In [7]:
def test(model,test_dataloader,criterion):
#test
    y_true = []
    y_pred = []
    test_total=0
    with torch.no_grad():
        ep_test_loss=0
        ep_test_acc=0
        for batch in tqdm(test_dataloader):
            prem, hyp, label = batch
            prem, hyp, label= prem.to(device), hyp.to(device), label.to(device)
            output = model(prem, hyp)
            y_true.extend(label.cpu())
            y_pred.extend(output.cpu().argmax(1))
            loss = criterion(output, label)
            ep_test_loss+=loss.item()
            acc = (output.argmax(1) == label).sum().item()
            ep_test_acc+=acc
            test_total+=label.size(0)
        print("Test Loss: ",ep_test_loss/len(test_dataloader)," Test Accuracy: ",ep_test_acc/test_total)
        print(classification_report(y_true, y_pred, target_names=["entailment","neutral","contradiction"]))
        print(confusion_matrix(y_true, y_pred))
        


In [8]:
EMBEDDING_DIM = 300
HIDDEN_DIM = 100
OUTPUT_DIM = 3
N_FC_LAYERS = 2
DROPOUT = 0.3
EPOCHS = 30
lr=0.001

In [9]:
datasetName="mnli"

mtrain_dataloader,mdev_dataloader,mtest_dataloader,membedding_matrix,mword2index = prepData(datasetName)

INPUT_DIM = len(mword2index)

train data


100%|███████████████████████████████| 392702/392702 [00:02<00:00, 142146.51it/s]


dev data


100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 126808.85it/s]


test data


100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 124214.95it/s]


preprocess


100%|████████████████████████████████████| 9832/9832 [00:00<00:00, 18068.43it/s]
/tmp/ipykernel_67675/1833773205.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  premise = torch.tensor(dataset["premise"],dtype=torch.long)


In [10]:


#initialize the model with above parameters
mnli_model = NLI_LSTM(membedding_matrix,INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_FC_LAYERS, DROPOUT)
#adam optimizer
mnli_model.to(device)
optimizer = optim.Adam(mnli_model.parameters())
#loss function
criterion = nn.CrossEntropyLoss()
#train
mnli_model = train(mnli_model,mtrain_dataloader,mdev_dataloader,mtest_dataloader,optimizer,criterion,datasetName,EPOCHS)

#test
test(mnli_model,mtest_dataloader,criterion)




Epoch:  1
Training


100%|████████████████████████████████████| 12272/12272 [00:57<00:00, 214.83it/s]


Validation


100%|████████████████████████████████████████| 307/307 [00:00<00:00, 518.62it/s]


Test


...

Test


100%|████████████████████████████████████████| 308/308 [00:01<00:00, 267.31it/s]


Epoch:  30  Train Loss:  0.8556207113157389 Train Accuracy:  0.6148631786952957
Epoch:  30  Val Loss:  0.8984044167040225  Val Accuracy:  0.6069281711665817
Epoch:  30 Test Loss:  0.8809550278759622  Test Accuracy:  0.6084214808787632

Validation Classification report
               precision    recall  f1-score   support

   entailment       0.54      0.81      0.65      3479
      neutral       0.66      0.37      0.47      3123
contradiction       0.69      0.62      0.65      3213

     accuracy                           0.61      9815
    macro avg       0.63      0.60      0.59      9815
 weighted avg       0.63      0.61      0.59      9815

Test Classification report
               precision    recall  f1-score   support

   entailment       0.54      0.83      0.65      3463
      neutral       0.64      0.36      0.46      3129
contradiction       0.72      0.62      0.66      3240

     accuracy                           0.61      9832
    macro avg       0.63      0.60     

100%|████████████████████████████████████████| 308/308 [00:01<00:00, 294.98it/s]


Test Loss:  0.8827209006269257  Test Accuracy:  0.60913344182262
               precision    recall  f1-score   support

   entailment       0.54      0.83      0.65      3463
      neutral       0.64      0.36      0.46      3129
contradiction       0.72      0.62      0.66      3240

     accuracy                           0.61      9832
    macro avg       0.63      0.60      0.59      9832
 weighted avg       0.63      0.61      0.60      9832

[[2861  354  248]
 [1466 1130  533]
 [ 956  286 1998]]


In [11]:
datasetName="snli"

strain_dataloader,sdev_dataloader,stest_dataloader,sembedding_matrix,sword2index = prepData(datasetName)
#intialize the model
INPUT_DIM = len(sword2index)

train data


100%|████████████████████████████████| 550152/550152 [00:07<00:00, 75061.04it/s]


dev data


100%|██████████████████████████████████| 10000/10000 [00:00<00:00, 67383.68it/s]


test data


100%|██████████████████████████████████| 10000/10000 [00:00<00:00, 67923.51it/s]


preprocess


100%|████████████████████████████████████| 9824/9824 [00:00<00:00, 25865.49it/s]


In [12]:
#initialize the model with above parameters
snli_model = NLI_LSTM(sembedding_matrix,INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_FC_LAYERS, DROPOUT)
snli_model.to(device)
#adam optimizer
optimizer = optim.Adam(snli_model.parameters())
#loss function
criterion = nn.CrossEntropyLoss()
#train
snli_model = train(snli_model,strain_dataloader,sdev_dataloader,stest_dataloader,optimizer,criterion,datasetName,EPOCHS)

#test
test(snli_model,stest_dataloader,criterion)



Epoch:  1
Training


100%|████████████████████████████████████| 17168/17168 [01:16<00:00, 223.71it/s]


Validation


100%|████████████████████████████████████████| 308/308 [00:00<00:00, 455.40it/s]


Test


...

Test


100%|████████████████████████████████████████| 307/307 [00:00<00:00, 450.84it/s]


Epoch:  30  Train Loss:  0.6722821749352745 Train Accuracy:  0.7239167987884237
Epoch:  30  Val Loss:  0.669052271390116  Val Accuracy:  0.7260719365982524
Epoch:  30 Test Loss:  0.6685523192734982  Test Accuracy:  0.7290309446254072

Validation Classification report
               precision    recall  f1-score   support

   entailment       0.75      0.74      0.74      3329
      neutral       0.68      0.68      0.68      3235
contradiction       0.75      0.76      0.75      3278

     accuracy                           0.73      9842
    macro avg       0.73      0.73      0.73      9842
 weighted avg       0.73      0.73      0.73      9842

Test Classification report
               precision    recall  f1-score   support

   entailment       0.75      0.75      0.75      3368
      neutral       0.68      0.68      0.68      3219
contradiction       0.75      0.75      0.75      3237

     accuracy                           0.73      9824
    macro avg       0.73      0.73      

100%|████████████████████████████████████████| 307/307 [00:00<00:00, 448.05it/s]


Test Loss:  0.6656459579638627  Test Accuracy:  0.7292345276872965
               precision    recall  f1-score   support

   entailment       0.76      0.74      0.75      3368
      neutral       0.69      0.69      0.69      3219
contradiction       0.75      0.76      0.75      3237

     accuracy                           0.73      9824
    macro avg       0.73      0.73      0.73      9824
 weighted avg       0.73      0.73      0.73      9824

[[2481  568  319]
 [ 468 2231  520]
 [ 335  450 2452]]


In [13]:
INPUT_DIM = len(mword2index)
mnli_model=NLI_LSTM(membedding_matrix,INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_FC_LAYERS, DROPOUT)
mnli_model.to(device)
mnli_model.load_state_dict(torch.load("../models/model_lstm_nli_mnli_ep_1.pt"))


#adam optimizer
optimizer = optim.Adam(mnli_model.parameters())
#loss function
criterion = nn.CrossEntropyLoss()
# #train
# model = train(model,train_dataloader,dev_dataloader,test_dataloader,optimizer,criterion,datasetName,EPOCHS)

#test
test(mnli_model,mtest_dataloader,criterion)

100%|████████████████████████████████████████| 308/308 [00:01<00:00, 225.52it/s]


Test Loss:  1.0207105209301044  Test Accuracy:  0.47447111472742065
               precision    recall  f1-score   support

   entailment       0.54      0.24      0.33      3463
      neutral       0.46      0.44      0.45      3129
contradiction       0.46      0.76      0.58      3240

     accuracy                           0.47      9832
    macro avg       0.49      0.48      0.45      9832
 weighted avg       0.49      0.47      0.45      9832

[[ 835 1120 1508]
 [ 433 1374 1322]
 [ 265  519 2456]]


In [14]:
INPUT_DIM = len(sword2index)
snli_model=NLI_LSTM(sembedding_matrix,INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_FC_LAYERS, DROPOUT)
snli_model.to(device)
snli_model.load_state_dict(torch.load("../models/model_lstm_nli_snli_ep_10.pt"))


#adam optimizer
optimizer = optim.Adam(snli_model.parameters())
#loss function
criterion = nn.CrossEntropyLoss()
# #train
# model = train(model,train_dataloader,dev_dataloader,test_dataloader,optimizer,criterion,datasetName,EPOCHS)

#test
test(snli_model,stest_dataloader,criterion)

100%|████████████████████████████████████████| 307/307 [00:00<00:00, 375.48it/s]


Test Loss:  0.7142397785613902  Test Accuracy:  0.7032776872964169
               precision    recall  f1-score   support

   entailment       0.68      0.81      0.74      3368
      neutral       0.77      0.53      0.62      3219
contradiction       0.69      0.77      0.73      3237

     accuracy                           0.70      9824
    macro avg       0.71      0.70      0.70      9824
 weighted avg       0.71      0.70      0.70      9824

[[2725  255  388]
 [ 796 1693  730]
 [ 484  262 2491]]
